# Plan: iWildCam 2020 - FGVC7

Objectives:
- Establish GPU environment check and install PyTorch cu121 if needed
- Load/train split from provided JSONs and verify sample_submission format
- Build a fast, correct baseline (transfer learning, torchvision model) with deterministic CV
- Iterate with augmentations, class-balancing, and inference-time tricks
- Save submission.csv and aim for medal CV and LB

Data understanding:
- train/: 157,199 images
- test/: 60,760 images
- iwildcam2020_train_annotations.json: labels/metadata
- iwildcam2020_test_information.json: test metadata
- iwildcam2020_megadetector_results.json: detections (optional ROI crop feature)
- sample_submission.csv: required format

Validation strategy:
- Stratified Group KFold by location/camera if available (mirror domain shift); otherwise by site/camera in annotations
- Single seeded split to start; later multi-seed if time

Baseline model:
- Torchvision pretrained CNN (e.g., EfficientNet-B3/B4 or ResNet50) finetune
- Mixed precision, cosine LR, label smoothing, balanced sampler or weighted loss
- Resize ~384px; augmentations (flip, color jitter, random resized crop)
- Early stop via fixed epochs (e.g., 6-10) due to time; monitor CV accuracy

Speed tactics:
- Verify GPU with nvidia-smi
- Cache dataloaders with fast jpeg decoder (torchvision.io or pillow-simd if available)
- Subsample smoke test (1k images, 1 epoch) before full run
- Log fold index, elapsed time, and periodic progress

Roadmap (with expert checkpoints):
1) Env check + dataset inspection (request expert review after plan) 
2) Implement data pipeline + CV split
3) Smoke train baseline
4) Full train baseline → generate submission
5) Analyze OOF errors; add improvements: class weights, stronger aug, TTA, center-crop on MegaDetector boxes
6) Blend seeds/models if time

Risks & mitigations:
- Class imbalance: use weighted CE or focal loss; balanced sampler
- Domain shift: location-aware CV; robust aug
- Time: keep runs short, monitor, and avoid long hyperparam sweeps

Next:
- Run GPU check and quick data profile
- Then request expert review of the plan and initial approach

In [1]:
# Environment/GPU check, deps, and quick data profile
import os, sys, json, time, shutil, subprocess, gc
from pathlib import Path
import pandas as pd

def run(cmd):
    print('>>>', ' '.join(cmd), flush=True)
    return subprocess.run(cmd, check=False, text=True, capture_output=True)

t0 = time.time()
print('Checking GPU with nvidia-smi...', flush=True)
res = run(['bash','-lc','nvidia-smi || true'])
print(res.stdout)

# Install exact cu121 torch stack if needed
def ensure_torch_cu121():
    try:
        import torch
        ok = (getattr(torch.version,'cuda','') or '').startswith('12.1') and torch.cuda.is_available()
        print('Existing torch:', torch.__version__, 'CUDA build:', getattr(torch.version,'cuda',None), 'CUDA available:', torch.cuda.is_available())
        if ok:
            return
        else:
            print('Reinstalling correct torch stack (cu121)...')
    except Exception as e:
        print('Torch import failed, installing cu121 stack...', e)
    # Uninstall any existing torch stack
    for pkg in ('torch','torchvision','torchaudio'):
        subprocess.run([sys.executable,'-m','pip','uninstall','-y',pkg], check=False)
    # Clean stray site dirs
    for d in (
        '/app/.pip-target/torch','/app/.pip-target/torchvision','/app/.pip-target/torchaudio',
        '/app/.pip-target/torchgen','/app/.pip-target/functorch',
    ):
        if os.path.exists(d):
            print('Removing', d)
            shutil.rmtree(d, ignore_errors=True)
    # Install cu121 stack
    subprocess.run([sys.executable,'-m','pip','install','--index-url','https://download.pytorch.org/whl/cu121','--extra-index-url','https://pypi.org/simple','torch==2.4.1','torchvision==0.19.1','torchaudio==2.4.1'], check=True)
    # constraints file
    Path('constraints.txt').write_text('torch==2.4.1\ntorchvision==0.19.1\ntorchaudio==2.4.1\n')
    import torch
    print('torch:', torch.__version__, 'built CUDA:', getattr(torch.version,'cuda',None))
    print('CUDA available:', torch.cuda.is_available())
    assert (getattr(torch.version,'cuda','') or '').startswith('12.1'), f'Wrong CUDA build: {torch.version.cuda}'
    assert torch.cuda.is_available(), 'CUDA not available'
    print('GPU:', torch.cuda.get_device_name(0))

ensure_torch_cu121()

# Install additional deps (timm, albumentations, opencv) respecting constraints, without upgrading torch
print('Installing additional dependencies if missing...', flush=True)
def pip_install_pkgs(pkgs):
    args = [sys.executable,'-m','pip','install','-c','constraints.txt',*pkgs,'--upgrade-strategy','only-if-needed'] if Path('constraints.txt').exists() else [sys.executable,'-m','pip','install',*pkgs,'--upgrade-strategy','only-if-needed']
    run(args)
try:
    import timm  # noqa
except Exception:
    pip_install_pkgs(['timm==1.0.9'])
try:
    import albumentations as A  # noqa
except Exception:
    pip_install_pkgs(['albumentations==1.4.14'])
try:
    import cv2  # noqa
except Exception:
    pip_install_pkgs(['opencv-python-headless'])

print('Loading dataset JSONs...', flush=True)
train_json = json.loads(Path('iwildcam2020_train_annotations.json').read_text())
test_json = json.loads(Path('iwildcam2020_test_information.json').read_text())
md_json = json.loads(Path('iwildcam2020_megadetector_results.json').read_text())

# Basic structure checks
images = {im['id']: im for im in train_json.get('images', [])}
ann = train_json.get('annotations', [])
categories = train_json.get('categories', [])
cat_ids = [c['id'] for c in categories]
locs = [im.get('location') for im in images.values()]
print(f"Train images: {len(images)} | Annotations: {len(ann)} | Categories: {len(categories)} | Unique locations: {len(set(locs))}")

# Build label mapping: category_id <-> index
cat_id_to_idx = {cid:i for i, cid in enumerate(sorted(cat_ids))}
idx_to_cat_id = {i:cid for cid,i in cat_id_to_idx.items()}
print('First 5 category_id->idx:', list(cat_id_to_idx.items())[:5])
Path('label_mapping.json').write_text(json.dumps({'cat_id_to_idx': cat_id_to_idx, 'idx_to_cat_id': idx_to_cat_id}))

# Build dataframe with image-level info
rows = []
for a in ann:
    img = images.get(a['image_id'])
    if img is None:
        continue
    rows.append({
        'image_id': a['image_id'],
        'file_name': img.get('file_name'),
        'location': img.get('location'),
        'seq_id': img.get('seq_id', None),
        'category_id': a['category_id'],
        'label_idx': cat_id_to_idx[a['category_id']],
    })
df = pd.DataFrame(rows)
print(df.head())
print('Label distribution (top 10):')
print(df['category_id'].value_counts().head(10))
print('Locations per fold candidate (sample):', df['location'].value_counts().head(10).to_dict())

# Test info and sample submission checks
ss = pd.read_csv('sample_submission.csv')
print('Sample submission columns:', ss.columns.tolist(), 'shape:', ss.shape)
print(ss.head())

test_images = test_json.get('images', [])
print('Test images in JSON:', len(test_images), '| Test files on disk:', len(list(Path('test').glob('*.jpg'))))

# Map between test Id and expected filename if possible
test_df = pd.DataFrame(test_images)
print('Test JSON columns:', test_df.columns.tolist())
if 'id' in test_df.columns:
    missing_ids = set(ss['Id']) - set(test_df['id'])
    extra_ids = set(test_df['id']) - set(ss['Id'])
    print('Submission Ids missing in test_json:', len(missing_ids), '| extra in json not in sample:', len(extra_ids))

# MegaDetector quick stats
md_images = md_json.get('images', []) if isinstance(md_json, dict) else []
print('MegaDetector entries:', len(md_images))
if md_images:
    # Count images with at least one bbox above 0.3
    cnt = 0
    for r in md_images[:5000]:
        dets = r.get('detections', [])
        if any(d.get('conf', d.get('confidence', 0)) >= 0.3 for d in dets):
            cnt += 1
    print('MD sample (5k) with conf>=0.3:', cnt)

print(f'Env+data profile done in {time.time()-t0:.1f}s')

Checking GPU with nvidia-smi...


>>> bash -lc nvidia-smi || true


Sat Sep 27 06:06:04 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.144.06             Driver Version: 550.144.06     CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA A10-24Q                 On  |   00000002:00:00.0 Off |                    0 |
| N/A   N/A    P0             N/A /  N/A  |     182MiB /  24512MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

Looking in indexes: https://download.pytorch.org/whl/cu121, https://pypi.org/simple


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 799.0/799.0 MB 423.7 MB/s eta 0:00:00


...

>>> /usr/bin/python3.11 -m pip install -c constraints.txt timm==1.0.9 --upgrade-strategy only-if-needed


>>> /usr/bin/python3.11 -m pip install -c constraints.txt albumentations==1.4.14 --upgrade-strategy only-if-needed


Loading dataset JSONs...


Train images: 157199 | Annotations: 157199 | Categories: 267 | Unique locations: 225
First 5 category_id->idx: [(0, 0), (2, 1), (3, 2), (4, 3), (6, 4)]


                               image_id  \
0  96b00332-21bc-11ea-a13a-137349068a90   
1  9017f7aa-21bc-11ea-a13a-137349068a90   
2  90d93c58-21bc-11ea-a13a-137349068a90   
3  887cd0ec-21bc-11ea-a13a-137349068a90   
4  9137d902-21bc-11ea-a13a-137349068a90   

                                  file_name  location  \
0  96b00332-21bc-11ea-a13a-137349068a90.jpg       267   
1  9017f7aa-21bc-11ea-a13a-137349068a90.jpg        45   
2  90d93c58-21bc-11ea-a13a-137349068a90.jpg        45   
3  887cd0ec-21bc-11ea-a13a-137349068a90.jpg        45   
4  9137d902-21bc-11ea-a13a-137349068a90.jpg       457   

                                 seq_id  category_id  label_idx  
0  96b004ea-21bc-11ea-a13a-137349068a90           73         34  
1  9017f8cc-21bc-11ea-a13a-137349068a90          227        105  
2  90d93e38-21bc-11ea-a13a-137349068a90          250        115  
3  887cd29a-21bc-11ea-a13a-137349068a90            2          1  
4  9137da9c-21bc-11ea-a13a-137349068a90          233        108  
La

In [2]:
# Build GroupKFold by location and prep MegaDetector (MD) detections maps
import json, math, re
from pathlib import Path
import pandas as pd
from sklearn.model_selection import GroupKFold

# Reconstruct df if not in memory
if 'df' not in globals():
    train_json = json.loads(Path('iwildcam2020_train_annotations.json').read_text())
    images = {im['id']: im for im in train_json.get('images', [])}
    ann = train_json.get('annotations', [])
    categories = train_json.get('categories', [])
    cat_ids = [c['id'] for c in categories]
    cat_id_to_idx = {cid:i for i, cid in enumerate(sorted(cat_ids))}
    rows = []
    for a in ann:
        img = images.get(a['image_id'])
        if img is None:
            continue
        rows.append({
            'image_id': a['image_id'],
            'file_name': img.get('file_name'),
            'location': img.get('location'),
            'seq_id': img.get('seq_id', None),
            'category_id': a['category_id'],
            'label_idx': cat_id_to_idx[a['category_id']],
        })
    df = pd.DataFrame(rows)
    print('Rebuilt train df:', df.shape)

# Create 5-fold GroupKFold by location
df = df.sample(frac=1.0, random_state=42).reset_index(drop=True)  # shuffle for nicer distribution
gkf = GroupKFold(n_splits=5)
folds = [-1]*len(df)
for fold, (trn_idx, val_idx) in enumerate(gkf.split(df, groups=df['location'])):
    for i in val_idx:
        folds[i] = fold
df['fold'] = folds
assert all(f>=0 for f in folds), 'Some folds unassigned'
print('Fold sizes:', df['fold'].value_counts().sort_index().to_dict())
print('Locations per fold (sample):')
print(df.groupby('fold')['location'].nunique())

# Save folds
out_folds = 'train_folds.parquet'
df.to_parquet(out_folds, index=False)
print('Saved', out_folds, Path(out_folds).stat().st_size, 'bytes')

# Prepare MegaDetector detections mapping
md = json.loads(Path('iwildcam2020_megadetector_results.json').read_text())
md_images = md.get('images', []) if isinstance(md, dict) else []
print('MD images entries:', len(md_images))

def norm_det_record(rec):
    # Expected fields: 'file' or 'image', 'detections': [{'category':'1','conf':0.9,'bbox':[x,y,w,h]}]
    fname = rec.get('file') or rec.get('image') or rec.get('file_name') or ''
    fname = Path(fname).name
    dets = rec.get('detections', [])
    out = []
    for d in dets:
        conf = d.get('conf', d.get('confidence', 0.0))
        bbox = d.get('bbox') or d.get('box')
        if bbox is None or len(bbox) != 4:
            continue
        # clamp bbox to [0,1] if they are normalized (as MD outputs are usually relative)
        x,y,w,h = bbox
        out.append({'conf': float(conf), 'bbox': [float(x), float(y), float(w), float(h)]})
    return fname, out

md_map = {}  # file_name -> list of dets
for rec in md_images:
    fname, dets = norm_det_record(rec)
    if not fname:
        continue
    md_map[fname] = dets
print('Built MD map for files:', len(md_map))

# Persist MD maps for fast access
with open('megadetector_map.json', 'w') as f:
    json.dump(md_map, f)
print('Saved megadetector_map.json')

# Quick sanity: count with >=0.3 conf
th = 0.3
has_obj = sum(1 for v in md_map.values() if any(d['conf']>=th for d in v))
print(f'MD files with conf>={th}:', has_obj)

print('Prep complete.')

Fold sizes: {0: 31440, 1: 31440, 2: 31440, 3: 31440, 4: 31439}
Locations per fold (sample):
fold
0    45
1    44
2    46
3    45
4    45
Name: location, dtype: int64
Saved train_folds.parquet 3902089 bytes


MD images entries: 280810


Built MD map for files: 0
Saved megadetector_map.json
MD files with conf>=0.3: 0
Prep complete.


In [3]:
# Inspect MegaDetector JSON structure to fix mapping
import json
from pathlib import Path

md = json.loads(Path('iwildcam2020_megadetector_results.json').read_text())
md_images = md.get('images', []) if isinstance(md, dict) else md
print('MD type:', type(md), '| images len:', len(md_images))
if md_images:
    print('Sample record keys:', list(md_images[0].keys()))
    # Show up to 5 records with file-like keys
    cnt_nonempty = 0
    for r in md_images[:50]:
        fname = r.get('file') or r.get('image') or r.get('file_name') or r.get('path') or ''
        if fname:
            cnt_nonempty += 1
            print('Example fname:', fname, '| detections:', len(r.get('detections', [])))
            break
    print('Non-empty fname count in first 50:', cnt_nonempty)
    # Count how many map to our train/test files by basename
    import pandas as pd
    train_names = set(pd.read_parquet('train_folds.parquet')['file_name'].unique())
    test_names = set([p.name for p in Path('test').glob('*.jpg')])
    matched = 0
    for r in md_images[:5000]:
        fname = r.get('file') or r.get('image') or r.get('file_name') or r.get('path') or ''
        base = Path(fname).name
        if base in train_names or base in test_names:
            matched += 1
    print('Basename matches in first 5k MD records:', matched)
else:
    print('No MD images found; check JSON format.')

MD type: <class 'dict'> | images len: 280810
Sample record keys: ['detections', 'id', 'max_detection_conf']
Non-empty fname count in first 50: 0
Basename matches in first 5k MD records: 0


In [4]:
# Rebuild MegaDetector mapping: records keyed by image 'id' not filename
import json
from pathlib import Path
import pandas as pd

train_json = json.loads(Path('iwildcam2020_train_annotations.json').read_text())
test_json = json.loads(Path('iwildcam2020_test_information.json').read_text())
md = json.loads(Path('iwildcam2020_megadetector_results.json').read_text())
md_images = md.get('images', []) if isinstance(md, dict) else md

# Build id->file_name lookup from train and test
train_id2fname = {im['id']: im['file_name'] for im in train_json.get('images', [])}
test_id2fname = {im['id']: im['file_name'] for im in test_json.get('images', [])}

def extract_dets(rec):
    dets = []
    for d in rec.get('detections', []):
        conf = d.get('conf', d.get('confidence', 0.0))
        bbox = d.get('bbox') or d.get('box')
        if bbox is None or len(bbox) != 4:
            continue
        x,y,w,h = bbox
        dets.append({'conf': float(conf), 'bbox': [float(x), float(y), float(w), float(h)]})
    return dets

md_by_id = {}   # image_id -> dets
fname_map = {}  # file_name -> dets
miss_id = 0
for rec in md_images:
    img_id = rec.get('id')
    if not img_id:
        continue
    dets = extract_dets(rec)
    md_by_id[img_id] = dets
    fname = train_id2fname.get(img_id) or test_id2fname.get(img_id)
    if fname:
        fname_map[fname] = dets
    else:
        miss_id += 1

with open('megadetector_by_id.json', 'w') as f:
    json.dump(md_by_id, f)
with open('megadetector_map.json', 'w') as f:
    json.dump(fname_map, f)

th = 0.3
has_obj_id = sum(1 for v in md_by_id.values() if any(d['conf']>=th for d in v))
has_obj_fname = sum(1 for v in fname_map.values() if any(d['conf']>=th for d in v))
print('MD images total:', len(md_images))
print('Mapped by id:', len(md_by_id), '| by filename:', len(fname_map), '| missing id->fname:', miss_id)
print(f'With conf>={th}: by id {has_obj_id}, by fname {has_obj_fname}')
print('MD mapping rebuild complete.')

MD images total: 280810
Mapped by id: 280810 | by filename: 217940 | missing id->fname: 62870
With conf>=0.3: by id 167640, by fname 134782
MD mapping rebuild complete.


In [5]:
# Dataset, transforms, and 1-epoch smoke train on a single fold with MD crop mix
import os, math, time, json, random, gc
from pathlib import Path
import numpy as np
import pandas as pd
import cv2
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import timm

torch.backends.cudnn.benchmark = True
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

SEED = 42
random.seed(SEED); np.random.seed(SEED); torch.manual_seed(SEED); torch.cuda.manual_seed_all(SEED)

# Load artifacts
df = pd.read_parquet('train_folds.parquet')
maps = json.loads(Path('label_mapping.json').read_text())
cat_id_to_idx = {int(k):int(v) for k,v in maps['cat_id_to_idx'].items()}
idx_to_cat_id = {int(k):int(v) for k,v in maps['idx_to_cat_id'].items()}
md_fname_map = json.loads(Path('megadetector_map.json').read_text()) if Path('megadetector_map.json').exists() else {}

num_classes = len(idx_to_cat_id)
img_size = 384
train_fold = 0  # use folds != train_fold for training, == for val in this smoke

def build_class_weights(df):
    counts = df['label_idx'].value_counts().sort_index().reindex(range(num_classes)).fillna(0).values.astype(np.float32)
    # 1/log(1.02+freq) smoothing
    weights = 1.0 / np.log(1.02 + counts + 1e-9)
    weights = weights / weights.mean()
    return torch.tensor(weights, dtype=torch.float32)

class IwDataset(Dataset):
    def __init__(self, df, root='train', train=True, md_map=None, p_crop=0.6):
        self.df = df.reset_index(drop=True)
        self.root = Path(root)
        self.train = train
        self.md_map = md_map or {}
        self.p_crop = p_crop

    def __len__(self):
        return len(self.df)

    def _apply_md_crop(self, img, file_name):
        dets = self.md_map.get(file_name, [])
        # choose largest bbox above threshold
        th = 0.3
        dets2 = [d for d in dets if d.get('conf', 0.0) >= th]
        if not dets2:
            return img
        # bbox in [x,y,w,h] normalized
        H, W = img.shape[:2]
        # compute area and choose largest
        def area(bb):
            return max(0.0, bb[2]) * max(0.0, bb[3])
        dets2.sort(key=lambda d: area(d['bbox']), reverse=True)
        x,y,w,h = dets2[0]['bbox']
        # expand
        pad = 0.15
        cx = x + w/2.0; cy = y + h/2.0
        side = max(w, h) * (1.0 + pad)
        x0 = max(0.0, cx - side/2.0); y0 = max(0.0, cy - side/2.0)
        x1 = min(1.0, cx + side/2.0); y1 = min(1.0, cy + side/2.0)
        # convert to pixels
        X0 = int(round(x0 * W)); Y0 = int(round(y0 * H)); X1 = int(round(x1 * W)); Y1 = int(round(y1 * H))
        if X1 <= X0 or Y1 <= Y0:
            return img
        # discard tiny crops (<4% area)
        if (X1 - X0) * (Y1 - Y0) < 0.04 * (W * H):
            return img
        return img[Y0:Y1, X0:X1, :]

    def _read_image(self, file_name):
        p = (self.root / file_name)
        img = cv2.imread(str(p))
        if img is None:
            # try absolute paths if any issue
            img = cv2.imdecode(np.fromfile(str(p), dtype=np.uint8), cv2.IMREAD_COLOR)
        if img is None:
            raise FileNotFoundError(str(p))
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        return img

    def _augment(self, img):
        if self.train:
            # random resized crop-like: choose random scale and center crop to square, then resize
            H, W = img.shape[:2]
            scale = random.uniform(0.7, 1.0)
            side = int(min(H, W) * scale)
            if side > 0:
                y0 = random.randint(0, max(0, H - side))
                x0 = random.randint(0, max(0, W - side))
                img = img[y0:y0+side, x0:x0+side]
            # random hflip
            if random.random() < 0.5:
                img = img[:, ::-1].copy()
            # light color jitter
            if random.random() < 0.8:
                # brightness/contrast
                alpha = 1.0 + random.uniform(-0.2, 0.2)
                beta = random.uniform(-20, 20)
                img = cv2.convertScaleAbs(img, alpha=alpha, beta=beta)
        # resize to square
        img = cv2.resize(img, (img_size, img_size), interpolation=cv2.INTER_AREA)
        return img

    def __getitem__(self, idx):
        r = self.df.iloc[idx]
        file_name = r['file_name']
        img = self._read_image(file_name)
        # optionally MD crop in training
        if self.train and random.random() < self.p_crop:
            img = self._apply_md_crop(img, file_name)
        img = self._augment(img)
        img = img.astype(np.float32) / 255.0
        # normalize with ImageNet stats
        mean = np.array([0.485, 0.456, 0.406], dtype=np.float32)
        std = np.array([0.229, 0.224, 0.225], dtype=np.float32)
        img = (img - mean) / std
        img = np.transpose(img, (2,0,1))  # CHW
        y = int(r['label_idx'])
        return torch.from_numpy(img), torch.tensor(y, dtype=torch.long)

# Split
trn_df = df[df['fold'] != train_fold].copy()
val_df = df[df['fold'] == train_fold].copy().reset_index(drop=True)
print('Train/Val shapes:', trn_df.shape, val_df.shape)

# Datasets and loaders
train_ds = IwDataset(trn_df, root='train', train=True, md_map=md_fname_map, p_crop=0.6)
val_ds = IwDataset(val_df, root='train', train=False, md_map=md_fname_map, p_crop=0.0)

bs = 32
train_loader = DataLoader(train_ds, batch_size=bs, shuffle=True, num_workers=6, pin_memory=True, persistent_workers=True)
val_loader = DataLoader(val_ds, batch_size=bs, shuffle=False, num_workers=6, pin_memory=True, persistent_workers=True)

# Model
model_name = 'tf_efficientnet_b4_ns'
model = timm.create_model(model_name, pretrained=True, num_classes=num_classes)
model = model.to(device).to(memory_format=torch.channels_last)

lr = 2e-4; wd = 1e-4
optimizer = torch.optim.AdamW(model.parameters(), lr=lr, weight_decay=wd)
lr_sched = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=1)  # for smoke, redefined later

class_weights = build_class_weights(trn_df).to(device)
criterion = nn.CrossEntropyLoss(weight=class_weights, label_smoothing=0.1).to(device)

scaler = torch.cuda.amp.GradScaler(enabled=True)

def evaluate(loader):
    model.eval()
    correct = 0; total = 0
    t0 = time.time()
    with torch.no_grad():
        for i, (x, y) in enumerate(loader):
            x = x.to(device, non_blocking=True).to(memory_format=torch.channels_last)
            y = y.to(device, non_blocking=True)
            with torch.cuda.amp.autocast(True):
                logits = model(x)
            preds = logits.argmax(dim=1)
            correct += (preds == y).sum().item()
            total += y.numel()
            if (i+1) % 100 == 0:
                print(f'  Eval step {i+1}/{len(loader)} elapsed {time.time()-t0:.1f}s', flush=True)
    acc = correct / max(1,total)
    return acc

# Smoke train: 1 epoch over a limited number of steps
max_train_steps = 300  # limit for smoke
print('Starting smoke training...')
t0 = time.time()
model.train()
step = 0
for i, (x, y) in enumerate(train_loader):
    x = x.to(device, non_blocking=True).to(memory_format=torch.channels_last)
    y = y.to(device, non_blocking=True)
    optimizer.zero_grad(set_to_none=True)
    with torch.cuda.amp.autocast(True):
        logits = model(x)
        loss = criterion(logits, y)
    scaler.scale(loss).backward()
    scaler.step(optimizer)
    scaler.update()
    step += 1
    if step % 20 == 0:
        print(f'  step {step} loss {loss.item():.4f} elapsed {time.time()-t0:.1f}s')
    if step >= max_train_steps:
        break
val_acc = evaluate(val_loader)
print(f'Smoke val acc: {val_acc:.4f} | elapsed {time.time()-t0:.1f}s')

# Save checkpoint for reuse
ckpt_path = f'ckpt_smoke_{model_name}_fold{train_fold}.pt'
torch.save({'model': model.state_dict(), 'val_acc': val_acc, 'cfg': {'model': model_name, 'img_size': img_size}}, ckpt_path)
print('Saved', ckpt_path)

/usr/local/lib/python3.11/dist-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Train/Val shapes: (125759, 7) (31440, 7)


/app/.pip-target/timm/models/_factory.py:117: UserWarning: Mapping deprecated model name tf_efficientnet_b4_ns to current tf_efficientnet_b4.ns_jft_in1k.
  model = create_fn(


Starting smoke training...


/tmp/ipykernel_196/3045649255.py:153: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = torch.cuda.amp.GradScaler(enabled=True)


...

Corrupt JPEG data: premature end of data segment
Corrupt JPEG data: premature end of data segment


Corrupt JPEG data: premature end of data segment


Corrupt JPEG data: premature end of data segment
Corrupt JPEG data: premature end of data segment


Corrupt JPEG data: premature end of data segment
Corrupt JPEG data: premature end of data segment


Smoke val acc: 0.5960 | elapsed 184.7s
Saved ckpt_smoke_tf_efficientnet_b4_ns_fold0.pt


In [6]:
# Inference: 4-view TTA (full + MD top1) with hflip; generate submission.csv
import os, json, time, math, random
from pathlib import Path
import numpy as np
import pandas as pd
import cv2
import torch
import timm

cv2.setNumThreads(0)
os.environ['OMP_NUM_THREADS'] = '1'
torch.backends.cuda.matmul.allow_tf32 = True
torch.backends.cudnn.allow_tf32 = True

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
img_size = 384
model_name = 'tf_efficientnet_b4_ns'

# Load mappings and test info
maps = json.loads(Path('label_mapping.json').read_text())
idx_to_cat_id = {int(k):int(v) for k,v in maps['idx_to_cat_id'].items()}
num_classes = len(idx_to_cat_id)
test_json = json.loads(Path('iwildcam2020_test_information.json').read_text())
test_images = test_json['images']
id2fname = {im['id']: im['file_name'] for im in test_images}
ss = pd.read_csv('sample_submission.csv')  # defines order and Id list

# MegaDetector filename map
md_fname_map = json.loads(Path('megadetector_map.json').read_text()) if Path('megadetector_map.json').exists() else {}

def read_rgb(path):
    img = cv2.imread(str(path))
    if img is None:
        img = cv2.imdecode(np.fromfile(str(path), dtype=np.uint8), cv2.IMREAD_COLOR)
    if img is None:
        raise FileNotFoundError(str(path))
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    return img

def md_top1_square_crop(img, file_name, conf_th=0.3, pad=0.20, min_area_ratio=0.04):
    dets = md_fname_map.get(file_name, [])
    dets = [d for d in dets if d.get('conf', 0.0) >= conf_th]
    if not dets:
        return img
    # choose largest area bbox
    dets.sort(key=lambda d: max(0.0, d['bbox'][2]) * max(0.0, d['bbox'][3]), reverse=True)
    x,y,w,h = dets[0]['bbox']  # normalized
    H,W = img.shape[:2]
    side = max(w, h) * (1.0 + pad)
    cx = x + w/2.0; cy = y + h/2.0
    x0 = max(0.0, cx - side/2.0); y0 = max(0.0, cy - side/2.0)
    x1 = min(1.0, cx + side/2.0); y1 = min(1.0, cy + side/2.0)
    X0 = int(round(x0 * W)); Y0 = int(round(y0 * H)); X1 = int(round(x1 * W)); Y1 = int(round(y1 * H))
    if X1 <= X0 or Y1 <= Y0:
        return img
    if (X1 - X0) * (Y1 - Y0) < min_area_ratio * (W * H):
        return img
    return img[Y0:Y1, X0:X1, :]

def preprocess(img):
    img = cv2.resize(img, (img_size, img_size), interpolation=cv2.INTER_AREA)
    img = img.astype(np.float32) / 255.0
    mean = np.array([0.485, 0.456, 0.406], dtype=np.float32)
    std = np.array([0.229, 0.224, 0.225], dtype=np.float32)
    img = (img - mean) / std
    img = np.transpose(img, (2,0,1))
    return torch.from_numpy(img)

# Load model checkpoint (use latest smoke or future full run)
ckpt_path = sorted(Path('.').glob('ckpt_smoke_*_fold0.pt'), key=lambda p: p.stat().st_mtime)[-1]
print('Loading checkpoint:', ckpt_path)
model = timm.create_model(model_name, pretrained=False, num_classes=num_classes)
state = torch.load(ckpt_path, map_location='cpu')
model.load_state_dict(state['model'], strict=False)
model = model.to(device).to(memory_format=torch.channels_last).eval()

all_preds = []
t0 = time.time()
with torch.no_grad():
    for i, id_ in enumerate(ss['Id'].tolist()):
        fname = id2fname[id_]
        path = Path('test') / fname
        img = read_rgb(path)
        crop = md_top1_square_crop(img, fname, conf_th=0.3, pad=0.20, min_area_ratio=0.04)
        # 4 views: full, hflip(full), crop, hflip(crop)
        views = []
        full = img
        views.append(preprocess(full))
        views.append(preprocess(full[:, ::-1, :].copy()))
        if crop is not None:
            views.append(preprocess(crop))
            views.append(preprocess(crop[:, ::-1, :].copy()))
        batch = torch.stack(views).to(device).to(memory_format=torch.channels_last)
        with torch.cuda.amp.autocast(True):
            logits = model(batch)
        probs = torch.softmax(logits, dim=1).mean(dim=0)  # average across views
        pred_idx = int(probs.argmax().item())
        pred_cat = idx_to_cat_id[pred_idx]
        all_preds.append(pred_cat)
        if (i+1) % 500 == 0:
            elapsed = time.time() - t0
            print(f'Processed {i+1}/{len(ss)} images, elapsed {elapsed/60:.1f} min')

sub = pd.DataFrame({'Id': ss['Id'], 'Category': all_preds})
sub.to_csv('submission.csv', index=False)
print('Saved submission.csv with shape', sub.shape)
print(sub.head())

Loading checkpoint: ckpt_smoke_tf_efficientnet_b4_ns_fold0.pt


/app/.pip-target/timm/models/_factory.py:117: UserWarning: Mapping deprecated model name tf_efficientnet_b4_ns to current tf_efficientnet_b4.ns_jft_in1k.
  model = create_fn(
/tmp/ipykernel_196/614934744.py:73: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Pl

/tmp/ipykernel_196/614934744.py:94: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(True):


Processed 500/60760 images, elapsed 0.8 min


Processed 1000/60760 images, elapsed 1.5 min


...

Processed 59000/60760 images, elapsed 81.5 min


Processed 59500/60760 images, elapsed 81.9 min


Processed 60000/60760 images, elapsed 82.4 min


Processed 60500/60760 images, elapsed 83.0 min


Saved submission.csv with shape (60760, 2)
                                     Id  Category
0  879d74d8-21bc-11ea-a13a-137349068a90         0
1  90243894-21bc-11ea-a13a-137349068a90         0
2  944adb30-21bc-11ea-a13a-137349068a90         0
3  8ced2424-21bc-11ea-a13a-137349068a90         0
4  8aac3a4c-21bc-11ea-a13a-137349068a90         0


In [8]:
# Full training loop: tf_efficientnet_b4_ns @384, 10 epochs, warmup+cosine, label smoothing, class weights
import os, time, json, random
from pathlib import Path
import numpy as np
import pandas as pd
import cv2
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset
import timm

cv2.setNumThreads(0)
os.environ['OMP_NUM_THREADS'] = '1'
torch.backends.cuda.matmul.allow_tf32 = True
torch.backends.cudnn.allow_tf32 = True
torch.backends.cudnn.benchmark = True

SEED = 42
random.seed(SEED); np.random.seed(SEED); torch.manual_seed(SEED); torch.cuda.manual_seed_all(SEED)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

img_size = 384
model_name = 'tf_efficientnet_b4_ns'
epochs = 10
train_fold = 0  # keep fold 0 as validation consistently
bs_train = 32
bs_val = 64
p_crop = 0.6
lr = 2e-4; wd = 1e-4

# Load artifacts
df = pd.read_parquet('train_folds.parquet')
maps = json.loads(Path('label_mapping.json').read_text())
cat_id_to_idx = {int(k):int(v) for k,v in maps['cat_id_to_idx'].items()}
idx_to_cat_id = {int(k):int(v) for k,v in maps['idx_to_cat_id'].items()}
num_classes = len(idx_to_cat_id)
md_fname_map = json.loads(Path('megadetector_map.json').read_text()) if Path('megadetector_map.json').exists() else {}

class IwDataset(Dataset):
    def __init__(self, df, root='train', train=True, md_map=None, p_crop=0.6):
        self.df = df.reset_index(drop=True)
        self.root = Path(root)
        self.train = train
        self.md_map = md_map or {}
        self.p_crop = p_crop if train else 0.0

    def __len__(self):
        return len(self.df)

    def _read_image(self, file_name):
        p = self.root / file_name
        img = cv2.imread(str(p))
        if img is None:
            buf = None
            try:
                buf = np.fromfile(str(p), dtype=np.uint8)
                img = cv2.imdecode(buf, cv2.IMREAD_COLOR)
            except Exception:
                img = None
        if img is None:
            # fallback to blank image to avoid DataLoader crash on missing/corrupt files
            img = np.zeros((img_size, img_size, 3), dtype=np.uint8)
        else:
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        return img

    def _apply_md_crop(self, img, file_name):
        dets = [d for d in self.md_map.get(file_name, []) if d.get('conf',0.0) >= 0.3]
        if not dets:
            return img
        dets.sort(key=lambda d: max(0.0,d['bbox'][2])*max(0.0,d['bbox'][3]), reverse=True)
        x,y,w,h = dets[0]['bbox']
        H,W = img.shape[:2]
        pad = 0.20
        side = max(w,h)*(1.0+pad)
        cx = x + w/2.0; cy = y + h/2.0
        x0 = max(0.0, cx - side/2.0); y0 = max(0.0, cy - side/2.0)
        x1 = min(1.0, cx + side/2.0); y1 = min(1.0, cy + side/2.0)
        X0 = int(round(x0*W)); Y0 = int(round(y0*H)); X1 = int(round(x1*W)); Y1 = int(round(y1*H))
        if X1<=X0 or Y1<=Y0:
            return img
        if (X1-X0)*(Y1-Y0) < 0.04*(W*H):
            return img
        return img[Y0:Y1, X0:X1, :]

    def _augment(self, img):
        if self.train:
            H,W = img.shape[:2]
            scale = random.uniform(0.7,1.0)
            side = int(min(H,W)*scale)
            if side>0 and H>0 and W>0:
                y0 = random.randint(0, max(0, H-side))
                x0 = random.randint(0, max(0, W-side))
                img = img[y0:y0+side, x0:x0+side]
            if random.random()<0.5:
                img = img[:, ::-1, :].copy()
            if random.random()<0.8:
                alpha = 1.0 + random.uniform(-0.2,0.2)
                beta = random.uniform(-20,20)
                img = cv2.convertScaleAbs(img, alpha=alpha, beta=beta)
        img = cv2.resize(img, (img_size,img_size), interpolation=cv2.INTER_AREA)
        return img

    def __getitem__(self, idx):
        r = self.df.iloc[idx]
        file_name = r['file_name']
        img = self._read_image(file_name)
        if self.train and random.random()<self.p_crop:
            img = self._apply_md_crop(img, file_name)
        img = self._augment(img).astype(np.float32)/255.0
        mean = np.array([0.485,0.456,0.406], dtype=np.float32)
        std = np.array([0.229,0.224,0.225], dtype=np.float32)
        img = (img-mean)/std
        img = np.transpose(img,(2,0,1))
        y = int(r['label_idx'])
        return torch.from_numpy(img), torch.tensor(y, dtype=torch.long)

def build_class_weights(df, num_classes):
    counts = df['label_idx'].value_counts().sort_index().reindex(range(num_classes)).fillna(0).values.astype(np.float32)
    w = 1.0/np.log(1.02+counts+1e-9)
    w = w/np.mean(w)
    return torch.tensor(w, dtype=torch.float32)

def run_training():
    # Filter to only existing image files to avoid DataLoader crashes
    train_files = set(p.name for p in Path('train').glob('*.jpg'))
    df_use = df[df['file_name'].isin(train_files)].copy()
    if len(df_use) < len(df):
        print(f'Filtered missing files: {len(df)-len(df_use)} removed, {len(df_use)} remain', flush=True)

    trn_df = df_use[df_use['fold'] != train_fold].copy()
    val_df = df_use[df_use['fold'] == train_fold].copy().reset_index(drop=True)
    print('Train/Val sizes:', trn_df.shape, val_df.shape, flush=True)

    train_ds = IwDataset(trn_df, root='train', train=True, md_map=md_fname_map, p_crop=p_crop)
    val_ds = IwDataset(val_df, root='train', train=False, md_map=md_fname_map, p_crop=0.0)

    train_loader = DataLoader(train_ds, batch_size=bs_train, shuffle=True, num_workers=8, pin_memory=True, persistent_workers=True, prefetch_factor=4)
    val_loader = DataLoader(val_ds, batch_size=bs_val, shuffle=False, num_workers=8, pin_memory=True, persistent_workers=True, prefetch_factor=4)

    model = timm.create_model(model_name, pretrained=True, num_classes=num_classes).to(device).to(memory_format=torch.channels_last)
    optimizer = torch.optim.AdamW(model.parameters(), lr=lr, weight_decay=wd)

    # Warmup + cosine
    num_steps = epochs*len(train_loader)
    warmup_steps = max(500, len(train_loader))
    def lr_lambda(step):
        if step < warmup_steps:
            return max(1e-3, step+1)/warmup_steps
        prog = (step - warmup_steps)/max(1, (num_steps - warmup_steps))
        return 0.5*(1+np.cos(np.pi*prog))
    scheduler = torch.optim.lr_scheduler.LambdaLR(optimizer, lr_lambda=lr_lambda)

    class_weights = build_class_weights(trn_df, num_classes).to(device)
    criterion = nn.CrossEntropyLoss(weight=class_weights, label_smoothing=0.1).to(device)
    scaler = torch.cuda.amp.GradScaler(enabled=True)

    best_acc = -1.0
    best_path = f'ckpt_best_{model_name}_fold{train_fold}.pt'
    global_step = 0
    t0 = time.time()
    for ep in range(1, epochs+1):
        model.train()
        ep_loss = 0.0
        t_ep = time.time()
        for i,(x,y) in enumerate(train_loader):
            x = x.to(device, non_blocking=True).to(memory_format=torch.channels_last)
            y = y.to(device, non_blocking=True)
            optimizer.zero_grad(set_to_none=True)
            with torch.cuda.amp.autocast(True):
                logits = model(x)
                loss = criterion(logits, y)
            scaler.scale(loss).backward()
            scaler.step(optimizer)
            scaler.update()
            scheduler.step()
            ep_loss += loss.item()
            global_step += 1
            if (i+1)%100==0:
                print(f'E{ep} step {i+1}/{len(train_loader)} loss {loss.item():.4f} lr {scheduler.get_last_lr()[0]:.2e} elapsed {time.time()-t_ep:.1f}s', flush=True)
        # eval
        model.eval()
        correct=0; total=0
        with torch.no_grad():
            for j,(x,y) in enumerate(val_loader):
                x = x.to(device, non_blocking=True).to(memory_format=torch.channels_last)
                y = y.to(device, non_blocking=True)
                with torch.cuda.amp.autocast(True):
                    logits = model(x)
                preds = logits.argmax(1)
                correct += (preds==y).sum().item()
                total += y.numel()
                if (j+1)%100==0:
                    print(f'  Val {j+1}/{len(val_loader)}', flush=True)
        acc = correct/max(1,total)
        print(f'End Epoch {ep}: loss {(ep_loss/max(1,len(train_loader))):.4f} val_acc {acc:.4f} epoch_time {time.time()-t_ep:.1f}s total {time.time()-t0:.1f}s', flush=True)
        if acc > best_acc:
            best_acc = acc
            torch.save({'model': model.state_dict(), 'val_acc': best_acc, 'cfg': {'model': model_name, 'img_size': img_size}}, best_path)
            print('  Saved best to', best_path, 'acc', best_acc, flush=True)
    print('Training done. Best acc:', best_acc, 'ckpt:', best_path, flush=True)

run_training()

Train/Val sizes: (125759, 7) (31440, 7)


/app/.pip-target/timm/models/_factory.py:117: UserWarning: Mapping deprecated model name tf_efficientnet_b4_ns to current tf_efficientnet_b4.ns_jft_in1k.
  model = create_fn(


/tmp/ipykernel_196/2920277601.py:143: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = torch.cuda.amp.GradScaler(enabled=True)


Corrupt JPEG data: premature end of data segment
Corrupt JPEG data: premature end of data segment


/tmp/ipykernel_196/2920277601.py:157: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(True):


...

Corrupt JPEG data: premature end of data segment


Corrupt JPEG data: premature end of data segment


Corrupt JPEG data: premature end of data segment


Corrupt JPEG data: premature end of data segment


FileNotFoundError: Caught FileNotFoundError in DataLoader worker process 6.
Original Traceback (most recent call last):
  File "/app/.pip-target/torch/utils/data/_utils/worker.py", line 309, in _worker_loop
    data = fetcher.fetch(index)  # type: ignore[possibly-undefined]
           ^^^^^^^^^^^^^^^^^^^^
  File "/app/.pip-target/torch/utils/data/_utils/fetch.py", line 52, in fetch
    data = [self.dataset[idx] for idx in possibly_batched_index]
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/app/.pip-target/torch/utils/data/_utils/fetch.py", line 52, in <listcomp>
    data = [self.dataset[idx] for idx in possibly_batched_index]
            ~~~~~~~~~~~~^^^^^
  File "/tmp/ipykernel_196/2920277601.py", line 100, in __getitem__
    img = self._read_image(file_name)
          ^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/tmp/ipykernel_196/2920277601.py", line 56, in _read_image
    raise FileNotFoundError(str(p))
FileNotFoundError: train/883572ba-21bc-11ea-a13a-137349068a9

In [7]:
# Validate and sanitize submission.csv before submit
import pandas as pd
import numpy as np
from pathlib import Path

ss = pd.read_csv('sample_submission.csv')
sub = pd.read_csv('submission.csv')
print('submission.csv shape:', sub.shape, '| expected:', ss.shape)
print('Columns:', sub.columns.tolist())

# Check ordering and Id coverage
same_ids = (sub['Id'].tolist() == ss['Id'].tolist())
print('Same Id order as sample:', same_ids)
missing = set(ss['Id']) - set(sub['Id'])
extra = set(sub['Id']) - set(ss['Id'])
print('Missing Ids:', len(missing), 'Extra Ids:', len(extra))

# Check Category dtype and NaNs
nan_cnt = sub['Category'].isna().sum()
print('NaN categories:', nan_cnt)
non_int = not pd.api.types.is_integer_dtype(sub['Category'])
print('Category integer dtype:', not non_int, '| dtype:', sub['Category'].dtype)

# Fixes if needed
if not same_ids or len(missing)>0 or len(extra)>0:
    sub = ss[['Id']].merge(sub, on='Id', how='left')
    print('Reordered/merged submission; new shape:', sub.shape)
if nan_cnt>0:
    # Fill NaNs with most frequent train category_id (often empty class id 0), safe fallback
    sub['Category'] = sub['Category'].fillna(0).astype(np.int64)
if non_int:
    # Coerce to int
    sub['Category'] = sub['Category'].astype(np.int64)

sub.to_csv('submission.csv', index=False)
print('Final submission.csv saved. Head:')
print(sub.head())
print('Value counts sample:', sub['Category'].value_counts().head(10))

submission.csv shape: (60760, 2) | expected: (60760, 2)
Columns: ['Id', 'Category']
Same Id order as sample: True
Missing Ids: 0 Extra Ids: 0
NaN categories: 0
Category integer dtype: True | dtype: int64
Final submission.csv saved. Head:
                                     Id  Category
0  879d74d8-21bc-11ea-a13a-137349068a90         0
1  90243894-21bc-11ea-a13a-137349068a90         0
2  944adb30-21bc-11ea-a13a-137349068a90         0
3  8ced2424-21bc-11ea-a13a-137349068a90         0
4  8aac3a4c-21bc-11ea-a13a-137349068a90         0
Value counts sample: Category
0      22476
71     11403
317     4486
90      3345
374     2914
372     2564
259     1407
96      1366
2        773
376      754
Name: count, dtype: int64


In [9]:
# Install pillow-simd and verify PIL loader performance settings
import os, sys, subprocess, shutil, time
from pathlib import Path

def run(cmd):
    print('>>>', ' '.join(cmd), flush=True)
    return subprocess.run(cmd, check=False, text=True, capture_output=True)

print('Uninstalling stock pillow (if any)...', flush=True)
run([sys.executable, '-m', 'pip', 'uninstall', '-y', 'pillow'])

print('Installing pillow-simd...', flush=True)
args = [sys.executable, '-m', 'pip', 'install']
if Path('constraints.txt').exists():
    args += ['-c', 'constraints.txt']
args += ['pillow-simd']
res = run(args)
print(res.stdout)
print(res.stderr)

print('Verifying PIL import and enabling truncated loads...', flush=True)
from PIL import Image, ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True
print('PIL version:', Image.__version__)

# Set recommended loader/thread env knobs
import cv2
cv2.setNumThreads(0)
os.environ['OMP_NUM_THREADS'] = '1'
print('cv2 threads set to 0; OMP_NUM_THREADS=1')
print('pillow-simd setup complete.')

Uninstalling stock pillow (if any)...


>>> /usr/bin/python3.11 -m pip uninstall -y pillow


Installing pillow-simd...


>>> /usr/bin/python3.11 -m pip install -c constraints.txt pillow-simd


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 904.6/904.6 KB 32.8 MB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Running setup.py clean for pillow-simd
Failed to build pillow-simd
  Running setup.py install for pillow-simd: started
  Running setup.py install for pillow-simd: finished with status 'error'

  error: subprocess-exited-with-error
  
  × python setup.py bdist_wheel did not run successfully.
  │ exit code: 1
  ╰─> [183 lines of output]
      running bdist_wheel
      running build
      running build_py
      creating build
      creating build/lib.linux-x86_64-3.11
      creating build/lib.linux-x86_64-3.11/PIL
      copying src/PIL/features.py -> build/lib.linux-x86_64-3.11/PIL
      copying src/PIL/_version.py -> build/lib.linux-x86_64-3.11/PIL
      copying src/PIL/_util.py -> build/lib.linux-x86_64-3.11/PIL
      copying src/PIL/_tkinter_finder.py -> build/lib.linux-x86_64-3.11/PIL
      copying src/

PIL version: 10.3.0
cv2 threads set to 0; OMP_NUM_THREADS=1
pillow-simd setup complete.


In [11]:
# Full training with PIL loader, top-2 MD crops, Mixup/CutMix, EMA, cosine schedule
import os, time, json, random, math
from pathlib import Path
import numpy as np
import pandas as pd
from PIL import Image, ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset
import timm
from timm.utils import ModelEmaV2
from timm.loss import SoftTargetCrossEntropy
from timm.data.mixup import Mixup

os.environ['OMP_NUM_THREADS'] = '1'
torch.backends.cuda.matmul.allow_tf32 = True
torch.backends.cudnn.allow_tf32 = True
torch.backends.cudnn.benchmark = True

SEED = 42
random.seed(SEED); np.random.seed(SEED); torch.manual_seed(SEED); torch.cuda.manual_seed_all(SEED)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

img_size = 384
model_name = 'tf_efficientnet_b4_ns'
epochs = 10
train_fold = 0  # fold 0 as validation
bs_train = 32
bs_val = 64
p_crop = 0.6
lr = 2e-4; wd = 1e-4
mixup_alpha = 0.1
cutmix_alpha = 0.5
mix_prob = 1.0
ema_decay = 0.9997

# Load artifacts
df = pd.read_parquet('train_folds.parquet')
maps = json.loads(Path('label_mapping.json').read_text())
cat_id_to_idx = {int(k):int(v) for k,v in maps['cat_id_to_idx'].items()}
idx_to_cat_id = {int(k):int(v) for k,v in maps['idx_to_cat_id'].items()}
num_classes = len(idx_to_cat_id)
md_fname_map = json.loads(Path('megadetector_map.json').read_text()) if Path('megadetector_map.json').exists() else {}

def build_class_weights(df, num_classes):
    counts = df['label_idx'].value_counts().sort_index().reindex(range(num_classes)).fillna(0).values.astype(np.float32)
    w = 1.0/np.log(1.02+counts+1e-9)
    w = w/np.mean(w)
    return torch.tensor(w, dtype=torch.float32)

def iou_xywh(a, b):
    ax, ay, aw, ah = a; bx, by, bw, bh = b
    ax0, ay0, ax1, ay1 = ax, ay, ax+aw, ay+ah
    bx0, by0, bx1, by1 = bx, by, bx+bw, by+bh
    ix0, iy0 = max(ax0, bx0), max(ay0, by0)
    ix1, iy1 = min(ax1, bx1), min(ay1, by1)
    iw, ih = max(0.0, ix1-ix0), max(0.0, iy1-iy0)
    inter = iw*ih
    ua = aw*ah + bw*bh - inter + 1e-9
    return inter/ua

class IwDataset(Dataset):
    def __init__(self, df, root='train', train=True, md_map=None, p_crop=0.6):
        self.df = df.reset_index(drop=True)
        self.root = Path(root)
        self.train = train
        self.md_map = md_map or {}
        self.p_crop = p_crop if train else 0.0

    def __len__(self):
        return len(self.df)

    def _read_image(self, file_name):
        p = self.root / file_name
        try:
            with Image.open(p) as im:
                im = im.convert('RGB')
                img = np.asarray(im)
        except Exception:
            # blank fallback
            img = np.zeros((img_size, img_size, 3), dtype=np.uint8)
        return img

    def _md_top2(self, file_name, conf_th=0.3, min_area=0.03, iou_th=0.7):
        dets = [d for d in self.md_map.get(file_name, []) if d.get('conf',0.0) >= conf_th]
        if not dets:
            return []
        dets.sort(key=lambda d: max(0.0,d['bbox'][2])*max(0.0,d['bbox'][3]), reverse=True)
        top = []
        for d in dets:
            if max(0.0,d['bbox'][2])*max(0.0,d['bbox'][3]) < min_area:
                continue
            ok = True
            for e in top:
                if iou_xywh(d['bbox'], e['bbox']) >= iou_th:
                    ok = False; break
            if ok:
                top.append(d)
            if len(top) >= 2:
                break
        return top

    def _apply_md_crop_square(self, img, bbox, pad=0.20):
        H, W = img.shape[:2]
        x,y,w,h = bbox
        side = max(w,h)*(1.0+pad)
        cx, cy = x + w/2.0, y + h/2.0
        x0 = max(0.0, cx - side/2.0); y0 = max(0.0, cy - side/2.0)
        x1 = min(1.0, cx + side/2.0); y1 = min(1.0, cy + side/2.0)
        X0 = int(round(x0*W)); Y0 = int(round(y0*H)); X1 = int(round(x1*W)); Y1 = int(round(y1*H))
        if X1<=X0 or Y1<=Y0:
            return img
        return img[Y0:Y1, X0:X1, :]

    def _augment(self, img):
        if self.train:
            H,W = img.shape[:2]
            if H>0 and W>0:
                scale = random.uniform(0.7,1.0)
                side = int(min(H,W)*scale)
                if side>0:
                    y0 = random.randint(0, max(0, H-side))
                    x0 = random.randint(0, max(0, W-side))
                    img = img[y0:y0+side, x0:x0+side]
            if random.random()<0.5:
                img = img[:, ::-1, :].copy()
            if random.random()<0.8:
                # brightness/contrast jitter
                alpha = 1.0 + random.uniform(-0.2, 0.2)
                beta = random.uniform(-20, 20)
                img = np.clip(img.astype(np.float32)*alpha + beta, 0, 255).astype(np.uint8)
        img = np.array(Image.fromarray(img).resize((img_size, img_size), resample=Image.BILINEAR))
        return img

    def __getitem__(self, idx):
        r = self.df.iloc[idx]
        file_name = r['file_name']
        img = self._read_image(file_name)
        if self.train and random.random() < self.p_crop:
            top = self._md_top2(file_name)
            if top:
                choice = random.choice(top)
                img = self._apply_md_crop_square(img, choice['bbox'], pad=0.20)
        img = self._augment(img).astype(np.float32)/255.0
        mean = np.array([0.485,0.456,0.406], dtype=np.float32)
        std = np.array([0.229,0.224,0.225], dtype=np.float32)
        img = (img-mean)/std
        img = np.transpose(img,(2,0,1))
        y = int(r['label_idx'])
        return torch.from_numpy(img), torch.tensor(y, dtype=torch.long)

def run_training(seed=42):
    random.seed(seed); np.random.seed(seed); torch.manual_seed(seed); torch.cuda.manual_seed_all(seed)
    # Filter existing files
    train_files = set(p.name for p in Path('train').glob('*.jpg'))
    df_use = df[df['file_name'].isin(train_files)].copy()
    if len(df_use) < len(df):
        print(f'Filtered missing files: {len(df)-len(df_use)} removed, {len(df_use)} remain', flush=True)
    trn_df = df_use[df_use['fold'] != train_fold].copy()
    val_df = df_use[df_use['fold'] == train_fold].copy().reset_index(drop=True)
    print('Train/Val sizes:', trn_df.shape, val_df.shape, flush=True)

    train_ds = IwDataset(trn_df, root='train', train=True, md_map=md_fname_map, p_crop=p_crop)
    val_ds = IwDataset(val_df, root='train', train=False, md_map=md_fname_map, p_crop=0.0)

    train_loader = DataLoader(train_ds, batch_size=bs_train, shuffle=True, num_workers=12, pin_memory=True, persistent_workers=True, prefetch_factor=4, drop_last=True)
    val_loader = DataLoader(val_ds, batch_size=bs_val, shuffle=False, num_workers=12, pin_memory=True, persistent_workers=True, prefetch_factor=4)

    model = timm.create_model(model_name, pretrained=True, num_classes=num_classes).to(device).to(memory_format=torch.channels_last)
    optimizer = torch.optim.AdamW(model.parameters(), lr=lr, weight_decay=wd)
    # EMA
    ema = ModelEmaV2(model, decay=ema_decay)

    # LR schedule per-step: 1 epoch warmup then cosine
    num_steps = epochs*len(train_loader)
    warmup_steps = max(1, len(train_loader))
    def lr_lambda(step):
        if step < warmup_steps:
            return (step+1)/warmup_steps
        prog = (step - warmup_steps)/max(1, (num_steps - warmup_steps))
        return 0.5*(1.0 + math.cos(math.pi*prog))
    scheduler = torch.optim.lr_scheduler.LambdaLR(optimizer, lr_lambda=lr_lambda)

    class_weights = build_class_weights(trn_df, num_classes).to(device)
    # Losses
    ce_weighted = nn.CrossEntropyLoss(weight=class_weights, label_smoothing=0.1).to(device)
    ce_soft = SoftTargetCrossEntropy().to(device)  # used during mixup/cutmix (no weights)

    # Mixup/CutMix
    mixup_fn = Mixup(mixup_alpha=mixup_alpha, cutmix_alpha=cutmix_alpha, cutmix_minmax=None, prob=mix_prob, switch_prob=0.0, mode='batch', label_smoothing=0.05, num_classes=num_classes)

    scaler = torch.cuda.amp.GradScaler(enabled=True)
    best_acc = -1.0
    best_path = f'ckpt_best_{model_name}_ema_fold{train_fold}_seed{seed}.pt'
    global_step = 0
    t0 = time.time()
    for ep in range(1, epochs+1):
        model.train()
        ep_loss = 0.0
        t_ep = time.time()
        mixing_active = ep <= max(1, epochs-2)  # disable in last 2 epochs
        for i,(x,y) in enumerate(train_loader):
            x = x.to(device, non_blocking=True).to(memory_format=torch.channels_last)
            y = y.to(device, non_blocking=True)
            optimizer.zero_grad(set_to_none=True)
            with torch.amp.autocast('cuda', enabled=True):
                if mixing_active and (x.size(0) % 2 == 0):
                    x_m, y_m = mixup_fn(x, y)
                    logits = model(x_m)
                    loss = ce_soft(logits, y_m)
                else:
                    logits = model(x)
                    loss = ce_weighted(logits, y)
            scaler.scale(loss).backward()
            scaler.step(optimizer)
            scaler.update()
            scheduler.step()
            # EMA update
            ema.update(model)
            ep_loss += loss.item()
            global_step += 1
            if (i+1)%200==0:
                print(f'E{ep} step {i+1}/{len(train_loader)} loss {loss.item():.4f} lr {scheduler.get_last_lr()[0]:.2e} elapsed {time.time()-t_ep:.1f}s', flush=True)
        # Evaluation (use EMA weights)
        correct=0; total=0
        eval_model = ema.module
        eval_model.eval()
        with torch.no_grad():
            for j,(xv,yv) in enumerate(val_loader):
                xv = xv.to(device, non_blocking=True).to(memory_format=torch.channels_last)
                yv = yv.to(device, non_blocking=True)
                with torch.amp.autocast('cuda', enabled=True):
                    logits = eval_model(xv)
                preds = logits.argmax(1)
                correct += (preds==yv).sum().item()
                total += yv.numel()
                if (j+1)%200==0:
                    print(f'  Val {j+1}/{len(val_loader)}', flush=True)
        acc = correct/max(1,total)
        print(f'End Epoch {ep}: loss {(ep_loss/max(1,len(train_loader))):.4f} val_acc {acc:.4f} epoch_time {time.time()-t_ep:.1f}s total {time.time()-t0:.1f}s', flush=True)
        if acc > best_acc:
            best_acc = acc
            torch.save({'model': eval_model.state_dict(), 'val_acc': best_acc, 'cfg': {'model': model_name, 'img_size': img_size, 'seed': seed}}, best_path)
            print('  Saved best EMA to', best_path, 'acc', best_acc, flush=True)
    print('Training done. Best acc:', best_acc, 'ckpt:', best_path, flush=True)

# Launch first run (fold 0, seed 42)
run_training(seed=42)

Train/Val sizes: (125759, 7) (31440, 7)


/app/.pip-target/timm/models/_factory.py:117: UserWarning: Mapping deprecated model name tf_efficientnet_b4_ns to current tf_efficientnet_b4.ns_jft_in1k.
  model = create_fn(


/tmp/ipykernel_196/1146100908.py:193: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = torch.cuda.amp.GradScaler(enabled=True)


E1 step 200/3929 loss 4.7494 lr 1.02e-05 elapsed 50.3s


E1 step 400/3929 loss 3.1894 lr 2.04e-05 elapsed 98.6s


...

E10 step 3800/3929 loss 74.9576 lr 6.57e-09 elapsed 929.8s


  Val 200/492


  Val 400/492


End Epoch 10: loss 72.6168 val_acc 0.7698 epoch_time 1041.2s total 10410.6s


Training done. Best acc: 0.8400763358778626 ckpt: ckpt_best_tf_efficientnet_b4_ns_ema_fold0_seed42.pt


In [ ]:
# Inference: 8-view TTA (full+hflip + top-2 MD crops+hflips), logit-average, sequence smoothing
import os, json, time, math, random
from pathlib import Path
import numpy as np
import pandas as pd
import cv2
import torch
import timm

cv2.setNumThreads(0)
os.environ['OMP_NUM_THREADS'] = '1'
torch.backends.cuda.matmul.allow_tf32 = True
torch.backends.cudnn.allow_tf32 = True

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
img_size = 384
model_name = 'tf_efficientnet_b4_ns'

# Load mappings and test info
maps = json.loads(Path('label_mapping.json').read_text())
idx_to_cat_id = {int(k):int(v) for k,v in maps['idx_to_cat_id'].items()}
num_classes = len(idx_to_cat_id)
test_json = json.loads(Path('iwildcam2020_test_information.json').read_text())
test_images = test_json['images']
id2fname = {im['id']: im['file_name'] for im in test_images}
id2seq = {im['id']: im.get('seq_id') for im in test_images}
ss = pd.read_csv('sample_submission.csv')  # defines order and Id list

# MegaDetector filename map
md_fname_map = json.loads(Path('megadetector_map.json').read_text()) if Path('megadetector_map.json').exists() else {}

def read_rgb(path):
    img = cv2.imread(str(path))
    if img is None:
        img = cv2.imdecode(np.fromfile(str(path), dtype=np.uint8), cv2.IMREAD_COLOR)
    if img is None:
        # fallback blank to avoid crash
        return np.zeros((img_size, img_size, 3), dtype=np.uint8)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    return img

def iou_xywh(a, b):
    ax, ay, aw, ah = a; bx, by, bw, bh = b
    ax0, ay0, ax1, ay1 = ax, ay, ax+aw, ay+ah
    bx0, by0, bx1, by1 = bx, by, bx+bw, by+bh
    ix0, iy0 = max(ax0, bx0), max(ay0, by0)
    ix1, iy1 = min(ax1, bx1), min(ay1, by1)
    iw, ih = max(0.0, ix1-ix0), max(0.0, iy1-iy0)
    inter = iw*ih
    ua = aw*ah + bw*bh - inter + 1e-9
    return inter/ua

def md_top2(file_name, conf_th=0.3, min_area=0.03, iou_th=0.7):
    dets = [d for d in md_fname_map.get(file_name, []) if d.get('conf',0.0) >= conf_th]
    if not dets:
        return []
    dets.sort(key=lambda d: max(0.0,d['bbox'][2])*max(0.0,d['bbox'][3]), reverse=True)
    top = []
    for d in dets:
        if max(0.0,d['bbox'][2])*max(0.0,d['bbox'][3]) < min_area:
            continue
        ok = True
        for e in top:
            if iou_xywh(d['bbox'], e['bbox']) >= iou_th:
                ok = False; break
        if ok:
            top.append(d)
        if len(top) >= 2:
            break
    return top

def md_square_crop(img, bbox, pad=0.20):
    H, W = img.shape[:2]
    x,y,w,h = bbox
    side = max(w,h)*(1.0+pad)
    cx = x + w/2.0; cy = y + h/2.0
    x0 = max(0.0, cx - side/2.0); y0 = max(0.0, cy - side/2.0)
    x1 = min(1.0, cx + side/2.0); y1 = min(1.0, cy + side/2.0)
    X0 = int(round(x0*W)); Y0 = int(round(y0*H)); X1 = int(round(x1*W)); Y1 = int(round(y1*H))
    if X1<=X0 or Y1<=Y0:
        return img
    return img[Y0:Y1, X0:X1, :]

def preprocess(img):
    img = cv2.resize(img, (img_size, img_size), interpolation=cv2.INTER_AREA)
    img = img.astype(np.float32) / 255.0
    mean = np.array([0.485, 0.456, 0.406], dtype=np.float32)
    std = np.array([0.229, 0.224, 0.225], dtype=np.float32)
    img = (img - mean) / std
    img = np.transpose(img, (2,0,1))
    return torch.from_numpy(img)

# Load best checkpoint (EMA-trained) if present; fallback to smoke
cand = sorted([*Path('.').glob('ckpt_best_*_ema_fold0_seed*.pt'), *Path('.').glob('ckpt_best_*_ema_fold0.pt'), *Path('.').glob('ckpt_smoke_*_fold0.pt')], key=lambda p: p.stat().st_mtime)
assert len(cand)>0, 'No checkpoint found'
ckpt_path = cand[-1]
print('Loading checkpoint:', ckpt_path)
model = timm.create_model(model_name, pretrained=False, num_classes=num_classes)
state = torch.load(ckpt_path, map_location='cpu')
model.load_state_dict(state['model'], strict=False)
model = model.to(device).to(memory_format=torch.channels_last).eval()

# Inference loop: collect per-image logits, then sequence-average logits
all_logits = []
seq_ids = []
t0 = time.time()
with torch.no_grad():
    for i, id_ in enumerate(ss['Id'].tolist()):
        fname = id2fname[id_]
        seq_ids.append(id2seq.get(id_))
        path = Path('test') / fname
        img = read_rgb(path)
        views = []
        # full + hflip
        full = img
        views.append(preprocess(full))
        views.append(preprocess(full[:, ::-1, :].copy()))
        # top-2 MD crops + hflips
        tops = md_top2(fname, conf_th=0.3, min_area=0.03, iou_th=0.7)
        for d in tops:
            crop = md_square_crop(img, d['bbox'], pad=0.20)
            views.append(preprocess(crop))
            views.append(preprocess(crop[:, ::-1, :].copy()))
        batch = torch.stack(views).to(device).to(memory_format=torch.channels_last)
        with torch.amp.autocast('cuda', enabled=True):
            logits = model(batch)
        # logit-average across views
        logits_mean = logits.mean(dim=0).float().cpu()
        all_logits.append(logits_mean.numpy())
        if (i+1) % 500 == 0:
            elapsed = time.time() - t0
            print(f'Processed {i+1}/{len(ss)} images, elapsed {elapsed/60:.1f} min', flush=True)

all_logits = np.stack(all_logits, axis=0)  # [N, C]
seq_ids = np.array(seq_ids)

# Sequence smoothing: average logits per seq_id
uniq_seqs = pd.factorize(seq_ids, sort=False)[0]
df_logits = pd.DataFrame(all_logits)
df_logits['seq_idx'] = uniq_seqs
seq_mean = df_logits.groupby('seq_idx').mean()  # averages logits per sequence
smoothed = seq_mean.loc[df_logits['seq_idx']].to_numpy()

pred_idx = smoothed.argmax(axis=1).astype(int)
pred_cat = [idx_to_cat_id[int(i)] for i in pred_idx]
sub = pd.DataFrame({'Id': ss['Id'], 'Category': pred_cat})
sub.to_csv('submission.csv', index=False)
print('Saved submission.csv with shape', sub.shape)
print(sub.head())

Loading checkpoint: ckpt_best_tf_efficientnet_b4_ns_ema_fold0_seed42.pt


/app/.pip-target/timm/models/_factory.py:117: UserWarning: Mapping deprecated model name tf_efficientnet_b4_ns to current tf_efficientnet_b4.ns_jft_in1k.
  model = create_fn(
/tmp/ipykernel_196/4075396547.py:99: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. P